In [1]:
# imports
import drgmum
from drgmum.toolkit.dataset import SmartDataset
from drgmum.chemdata.fingerprints import molprint2d_count_fingerprinter
from drgmum.chemdata.chembl.sqlite_queries import get_smiles
from drgmum.chemdata.utils.pandas_utils import two_level_series_to_csr
import logging
logging.basicConfig()
import os
data_folder = '/var/data/users/local/pocha/data'
chembl21db = os.path.join(data_folder, 'DRGMUM/chembl_21.db')
chembl_ids_file = os.path.join(data_folder, 'SCFP/Random_compounds_100.sdf')
similarity_matrix_file = os.path.join(data_folder, 'SCFP/Similarity150Dawid.csv')
folder_with_pairs = os.path.join(data_folder, 'SCFP/pairs')
from load_constraints import get_chembls, load_similarity_matrices, get_mapping
from sklearn.base import BaseEstimator, ClusterMixin
from sklearn.cluster import KMeans
from sklearn.utils.estimator_checks import check_estimator
from sklearn.model_selection import train_test_split
random_state = 666
from bidict import bidict
import numpy as np

In [2]:
# load data

# 1. load chembl IDs
chembl_ids = get_chembls(chembl_ids_file)
smiles = get_smiles(chembl21db, chembl_ids)

# 2. load fingerprints
a = molprint2d_count_fingerprinter(smiles) # nieważne
sprase_fp, row_labels, col_labels = two_level_series_to_csr(a)

# 3. load constraints
bin_sim, scale_sim, mapping_idx_chembl = load_similarity_matrices(similarity_matrix_file, chembl_ids_file, folder_with_pairs)
bin_sim_list = zip(zip(*bin_sim.nonzero()), bin_sim.data)
scale_sim_list = zip(zip(*scale_sim.nonzero()), scale_sim.data)

# 4. remove contraints that are duplicate (sim(x,y)=sim(y,x)) or
bin_sim_list = [x for x in bin_sim_list if x[0][0]<x[0][1]]
scale_sim_list = [x for x in scale_sim_list if x[0][0]<x[0][1]]

# 5. chcemy by mapowanie indeks-chembl dla sparse_fp i dla bin/scale_sim było takie samo
constraints_mapping = get_mapping(all_compounds_file=chembl_ids_file) # digit: chembl_id
fp_mapping = bidict(zip(row_labels, range(len(row_labels))))

constraints_2_fp_mapping = bidict([])
for chembl_id in row_labels:
    constraints_2_fp_mapping[constraints_mapping.inv[chembl_id]] = fp_mapping[chembl_id]

bin_sim_list = [((constraints_2_fp_mapping[x[0][0]], constraints_2_fp_mapping[x[0][1]]), x[1]) for x in bin_sim_list]
scale_sim_list = [((constraints_2_fp_mapping[x[0][0]], constraints_2_fp_mapping[x[0][1]]), x[1]) for x in scale_sim_list]

# 6. podział więzów na foldy
y_train, y_test = train_test_split(bin_sim_list, train_size=0.8, random_state=random_state)
# 343 - train, 86 - test 

INFO:drgmum.chemdata.chembl.sqlite_queries:Extracting SMILES from the database...
INFO:drgmum.chemdata.chembl.sqlite_queries:Done
INFO:drgmum.chemdata.fingerprints._openbabel:Processing 100 SMILES...
100it [00:00, 217772.79it/s]
645it [00:00, 347907.16it/s]
100%|██████████| 2144/2144 [00:00<00:00, 189533.11it/s]

21 pairs were omitted



/home/pocha/libs/anaconda2/lib/python2.7/site-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [3]:
# define score
def my_binary_score(labels_true, labels_pred):
    # labels true: ((x_i, x_j), sim(x_i, x_j))
    # labels_pred: list
    true, pred = np.array(labels_true), np.array(labels_pred)
    diffs = 0
    for ((idx_1, idx_2), sim_true) in labels_true:
        if labels_pred[idx_1] == labels_pred[idx_2]:
            diffs = diffs + (np.absolute(sim_true-1.))
        else:
            diffs = diffs + (np.absolute(sim_true-0.))
    return diffs/len(labels_true)

In [4]:
# define model (this is rather not needed)
# class KMeans_SCFP(KMeans):
#     def score():
#         pass
    
# check_estimator(KMeans_SCFP)

In [7]:
# define models and hiperparameters
hiperparameters = [{'n_clusters':1, 'init':'k-means++', 'n_init':1},
                   {'n_clusters':2, 'init':'k-means++', 'n_init':1},
                   {'n_clusters':3, 'init':'k-means++', 'n_init':1},
                   {'n_clusters':4, 'init':'k-means++', 'n_init':1},
                   {'n_clusters':5, 'init':'k-means++', 'n_init':1},
                   {'n_clusters':6, 'init':'k-means++', 'n_init':1},
                   {'n_clusters':7, 'init':'k-means++', 'n_init':1},
                   {'n_clusters':8, 'init':'k-means++', 'n_init':1},
                   {'n_clusters':9, 'init':'k-means++', 'n_init':1},
                   {'n_clusters':10, 'init':'k-means++', 'n_init':1}]


In [8]:
# define experiment
X = sprase_fp.toarray()
results = {}
for hips in hiperparameters:
    scores = []
    for i in range(5):
        model = KMeans(**hips)
        model.fit(X)
        predictions = model.predict(X)
        scores.append(my_binary_score(y_train, predictions))
    results[str(hips)]=np.mean(scores)
    
print results

{"{'init': 'k-means++', 'n_init': 1, 'n_clusters': 7}": 1.1411078717201166, "{'init': 'k-means++', 'n_init': 1, 'n_clusters': 4}": 1.3667638483965017, "{'init': 'k-means++', 'n_init': 1, 'n_clusters': 5}": 1.1889212827988338, "{'init': 'k-means++', 'n_init': 1, 'n_clusters': 8}": 1.068804664723032, "{'init': 'k-means++', 'n_init': 1, 'n_clusters': 9}": 1.082798833819242, "{'init': 'k-means++', 'n_init': 1, 'n_clusters': 2}": 1.5551020408163265, "{'init': 'k-means++', 'n_init': 1, 'n_clusters': 3}": 1.4816326530612245, "{'init': 'k-means++', 'n_init': 1, 'n_clusters': 1}": 1.7317784256559769, "{'init': 'k-means++', 'n_init': 1, 'n_clusters': 10}": 1.0466472303207, "{'init': 'k-means++', 'n_init': 1, 'n_clusters': 6}": 1.1959183673469389}
